<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/LangChain_AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary libraries
!pip install langchain-openai --quiet  # Installs a specific version (0.0.3) of the langchain_openai package quietly
!pip install langchain langchain-experimental --quiet        # Installs a specific version (0.1.1) of the langchain package quietly
!pip install huggingface_hub openai google-search-results tiktoken wikipedia --quiet

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "sk-cUxxxxxxxxxxxxtZ2sY"

## Zero-shot ReAct Agent

In [ ]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,
                 api_key=OPENAI_API_KEY)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)

from langchain.agents import Tool
tool_list = [Tool(name = "Math Tool", func=tools[0].run,
        description="Tool to calculate, nothing else",
        return_direct=True),]

from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, llm,
                         agent="zero-shot-react-description", verbose=True)
agent.run("What is 100 divided by 25?")



> Entering new AgentExecutor chain...
I can use the Math Tool to calculate the division.
Action: Math Tool
Action Input: 100 / 25
Observation: Answer: 4.0
Thought:I now know the final answer
Final Answer: 4.0

> Finished chain.


'4.0'

## COnversational ReAct Agent

In [ ]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, api_key=OPENAI_API_KEY)
tools = load_tools(["wikipedia"],  llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,tools=tools,
                                        llm=llm, verbose=False,  max_iterations=3, memory=memory)
output_1=conversational_agent.run("Who is this guy 'Elon Musk'?")
output_2=conversational_agent.run("which person we were taking about?")
print(output_1)
print("*"*100)
print(output_2)

Elon Reeve Musk, born June 28, 1971, is a prominent entrepreneur and business magnate known for his work in revolutionizing several industries. He is the founder, CEO, CTO, and chief designer of SpaceX; CEO and product architect of Tesla, Inc.; founder of The Boring Company; co-founder of Neuralink and OpenAI; and was involved in the creation of SolarCity. Musk is recognized for his role in the development of the electric vehicle market through Tesla and his ambitions in space exploration with SpaceX. He has also been involved in various controversies and has made headlines for his statements and actions on social media platforms. As of December 2023, he is considered the wealthiest person in the world according to the Bloomberg Billionaires Index and Forbes, with his wealth primarily derived from his stakes in Tesla and SpaceX.
****************************************************************************************************
We were talking about Elon Musk, the entrepreneur and busi

## Structured Chat Agent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage


# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
tools = load_tools(["wikipedia"],  llm=llm)
# Choose the LLM that will drive the agent
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106",api_key=OPENAI_API_KEY)
# Construct the JSON agent
agent = create_structured_chat_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke(
    {
        "input": "what's my name? Do not use tools unless you have to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Could not parse LLM output: I understand. Your name is Bob.Invalid or incomplete response{
  "action": "Final Answer",
  "action_input": "Your name is Bob."
}

> Finished chain.


{'input': "what's my name? Do not use tools unless you have to",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

# DocStore Agent

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
#binitializing an LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=OPENAI_API_KEY)

# initializing a docstore explorer
docstore = DocstoreExplorer(Wikipedia())

# initializing a tool
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to ask with search"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to ask with lookup")]

react_agent= initialize_agent(tools, llm, agent="react-docstore",verbose=True)
print(react_agent.run("What was Mahatma Gandhi's full name?")) # look on the keywords then go for search



> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find his full name.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.  
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for 21 years. It was here 

## Self Ask with Search Agent

In [ ]:
import os
os.environ['TAVILY_API_KEY'] = "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"

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilyAnswer
import os
os.environ['TAVILY_API_KEY'] = "tvly-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
tools = [TavilyAnswer(max_results=2, name="Intermediate Answer")]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=OPENAI_API_KEY)

# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is the reigning men's U.S. Open champion?The reigning men's U.S. Open champion is Daniil Medvedev. He won the tournament in 2023 by defeating Novak Djokovic in the final.Follow up: Where is Daniil Medvedev from?Daniil Medvedev is from Russia.So the final answer is: Russia

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Russia'}